In [ ]:
!pip install python-chess

In [ ]:
import chess
import chess.engine
import time
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from enum import Enum
import sys

class NodeType(Enum):
    EXACT = 1
    LOWER_BOUND = 2
    UPPER_BOUND = 3

@dataclass
class TranspositionEntry:
    depth: int
    score: int
    node_type: NodeType
    best_move: Optional[chess.Move]
    age: int

class AlphaBetaChessEngine:
    def __init__(self, max_depth: int = 5):
        self.max_depth = max_depth
        self.transposition_table: Dict[int, TranspositionEntry] = {}
        self.killer_moves: Dict[int, List[chess.Move]] = {}
        self.history_table: Dict[Tuple[int, int], int] = {}
        self.nodes_searched = 0
        self.tt_hits = 0
        self.pruning_count = 0
        self.current_age = 0

        # Initialize killer moves for each depth
        for depth in range(self.max_depth + 10):
            self.killer_moves[depth] = []

        # Piece values for evaluation
        self.piece_values = {
            chess.PAWN: 100,
            chess.KNIGHT: 320,
            chess.BISHOP: 330,
            chess.ROOK: 500,
            chess.QUEEN: 900,
            chess.KING: 20000
        }

        # Piece-Square Tables for positional evaluation
        self.pst = {
            chess.PAWN: [
                [  0,  0,  0,  0,  0,  0,  0,  0],
                [ 50, 50, 50, 50, 50, 50, 50, 50],
                [ 10, 10, 20, 30, 30, 20, 10, 10],
                [  5,  5, 10, 25, 25, 10,  5,  5],
                [  0,  0,  0, 20, 20,  0,  0,  0],
                [  5, -5,-10,  0,  0,-10, -5,  5],
                [  5, 10, 10,-20,-20, 10, 10,  5],
                [  0,  0,  0,  0,  0,  0,  0,  0]
            ],
            chess.KNIGHT: [
                [-50,-40,-30,-30,-30,-30,-40,-50],
                [-40,-20,  0,  0,  0,  0,-20,-40],
                [-30,  0, 10, 15, 15, 10,  0,-30],
                [-30,  5, 15, 20, 20, 15,  5,-30],
                [-30,  0, 15, 20, 20, 15,  0,-30],
                [-30,  5, 10, 15, 15, 10,  5,-30],
                [-40,-20,  0,  5,  5,  0,-20,-40],
                [-50,-40,-30,-30,-30,-30,-40,-50]
            ],
            chess.BISHOP: [
                [-20,-10,-10,-10,-10,-10,-10,-20],
                [-10,  0,  0,  0,  0,  0,  0,-10],
                [-10,  0,  5, 10, 10,  5,  0,-10],
                [-10,  5,  5, 10, 10,  5,  5,-10],
                [-10,  0, 10, 10, 10, 10,  0,-10],
                [-10, 10, 10, 10, 10, 10, 10,-10],
                [-10,  5,  0,  0,  0,  0,  5,-10],
                [-20,-10,-10,-10,-10,-10,-10,-20]
            ]
        }

    def evaluate_position(self, board: chess.Board) -> int:
        """
        Evaluate the current position from White's perspective
        """
        if board.is_checkmate():
            return -20000 if board.turn else 20000

        if board.is_stalemate() or board.is_insufficient_material():
            return 0

        score = 0

        # Material and positional evaluation
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is None:
                continue

            piece_value = self.piece_values[piece.piece_type]

            # Add positional bonus
            row, col = divmod(square, 8)
            if not piece.color:  # Black pieces
                row = 7 - row

            if piece.piece_type in self.pst:
                piece_value += self.pst[piece.piece_type][row][col]

            if piece.color:  # White
                score += piece_value
            else:  # Black
                score -= piece_value

        # Mobility bonus
        legal_moves = len(list(board.legal_moves))
        if board.turn:  # White to move
            score += legal_moves * 2
        else:  # Black to move
            score -= legal_moves * 2

        return score

    def is_quiescent(self, board: chess.Board) -> bool:
        """
        Check if position is quiet (no captures, checks, or promotions)
        """
        if board.is_check():
            return False

        for move in board.legal_moves:
            if board.is_capture(move) or move.promotion:
                return False

        return True

    def quiescence_search(self, board: chess.Board, alpha: int, beta: int, depth: int = 0) -> int:
        """
        Quiescence search to avoid horizon effect
        """
        self.nodes_searched += 1

        # Stand pat evaluation
        stand_pat = self.evaluate_position(board)

        if depth > 10:  # Prevent infinite quiescence
            return stand_pat

        if board.turn:  # Maximizing player (White)
            if stand_pat >= beta:
                return beta
            if stand_pat > alpha:
                alpha = stand_pat
        else:  # Minimizing player (Black)
            if stand_pat <= alpha:
                return alpha
            if stand_pat < beta:
                beta = stand_pat

        # Generate and search only captures and checks
        tactical_moves = []
        for move in board.legal_moves:
            if board.is_capture(move) or board.gives_check(move) or move.promotion:
                tactical_moves.append(move)

        # Order tactical moves (captures first, then checks)
        tactical_moves.sort(key=lambda m: (
            board.is_capture(m) * 1000 +
            (self.piece_values.get(board.piece_at(m.to_square).piece_type, 0) if board.piece_at(m.to_square) else 0) -
            self.piece_values.get(board.piece_at(m.from_square).piece_type, 0)
        ), reverse=True)

        for move in tactical_moves:
            board.push(move)
            score = self.quiescence_search(board, alpha, beta, depth + 1)
            board.pop()

            if board.turn:  # Maximizing player
                if score >= beta:
                    return beta
                if score > alpha:
                    alpha = score
            else:  # Minimizing player
                if score <= alpha:
                    return alpha
                if score < beta:
                    beta = score

        return alpha if board.turn else beta

    def order_moves(self, board: chess.Board, moves: List[chess.Move], depth: int, tt_move: Optional[chess.Move] = None) -> List[chess.Move]:
        """
        Order moves for better alpha-beta pruning:
        1. Transposition table move
        2. Captures (MVV-LVA)
        3. Killer moves
        4. History heuristic
        """
        def move_score(move):
            score = 0

            # Transposition table move gets highest priority
            if tt_move and move == tt_move:
                return 10000

            # Captures (Most Valuable Victim - Least Valuable Attacker)
            if board.is_capture(move):
                captured_piece = board.piece_at(move.to_square)
                capturing_piece = board.piece_at(move.from_square)
                if captured_piece and capturing_piece:
                    score += 1000 + self.piece_values[captured_piece.piece_type] - self.piece_values[capturing_piece.piece_type]

            # Promotions
            if move.promotion:
                score += 900

            # Checks
            if board.gives_check(move):
                score += 50

            # Killer moves
            if move in self.killer_moves.get(depth, []):
                score += 100

            # History heuristic
            move_key = (move.from_square, move.to_square)
            score += self.history_table.get(move_key, 0)

            return score

        return sorted(moves, key=move_score, reverse=True)

    def store_killer_move(self, move: chess.Move, depth: int):
        """Store killer move for move ordering"""
        if depth not in self.killer_moves:
            self.killer_moves[depth] = []

        if move not in self.killer_moves[depth]:
            self.killer_moves[depth].insert(0, move)
            if len(self.killer_moves[depth]) > 2:
                self.killer_moves[depth].pop()

    def update_history(self, move: chess.Move, depth: int):
        """Update history table for move ordering"""
        move_key = (move.from_square, move.to_square)
        self.history_table[move_key] = self.history_table.get(move_key, 0) + depth * depth

    def store_transposition(self, board: chess.Board, depth: int, score: int, node_type: NodeType, best_move: Optional[chess.Move]):
        """Store position in transposition table"""
        key = hash(board.fen())
        self.transposition_table[key] = TranspositionEntry(
            depth=depth,
            score=score,
            node_type=node_type,
            best_move=best_move,
            age=self.current_age
        )

    def probe_transposition(self, board: chess.Board, depth: int, alpha: int, beta: int) -> Tuple[Optional[int], Optional[chess.Move]]:
        """Probe transposition table for stored evaluation"""
        key = hash(board.fen())
        if key not in self.transposition_table:
            return None, None

        entry = self.transposition_table[key]
        self.tt_hits += 1

        if entry.depth >= depth:
            if entry.node_type == NodeType.EXACT:
                return entry.score, entry.best_move
            elif entry.node_type == NodeType.LOWER_BOUND and entry.score >= beta:
                return entry.score, entry.best_move
            elif entry.node_type == NodeType.UPPER_BOUND and entry.score <= alpha:
                return entry.score, entry.best_move

        return None, entry.best_move

    def alpha_beta_search(self, board: chess.Board, depth: int, alpha: int, beta: int, is_maximizing: bool) -> Tuple[int, Optional[chess.Move]]:
        """
        STEP-BY-STEP Alpha-Beta Pruning Implementation:

        Step 1: Initialize α and β (α = -∞, β = ∞)
        Step 2: Start at root and recursively explore each node
        Step 3: If max node, update α if better value found
                If min node, update β if better value found
        Step 4: If α ≥ β, stop exploring and prune (return best value)
        Step 5: Repeat until entire tree searched or pruned
        """

        self.nodes_searched += 1

        # Check transposition table first
        tt_score, tt_move = self.probe_transposition(board, depth, alpha, beta)
        if tt_score is not None:
            return tt_score, tt_move

        # Base case: maximum depth reached or game over
        if depth == 0 or board.is_game_over():
            if depth == 0 and not self.is_quiescent(board):
                # Enter quiescence search to avoid horizon effect
                score = self.quiescence_search(board, alpha, beta)
                return score, None
            return self.evaluate_position(board), None

        legal_moves = list(board.legal_moves)
        if not legal_moves:
            return self.evaluate_position(board), None

        # Order moves for better pruning
        ordered_moves = self.order_moves(board, legal_moves, depth, tt_move)

        best_move = None
        best_score = float('-inf') if is_maximizing else float('inf')
        original_alpha = alpha

        # STEP 2: Recursively explore each node
        for i, move in enumerate(ordered_moves):
            board.push(move)

            # Recursive call with swapped maximizing flag
            score, _ = self.alpha_beta_search(board, depth - 1, alpha, beta, not is_maximizing)

            board.pop()

            # STEP 3: Update α (max node) or β (min node)
            if is_maximizing:  # MAX NODE
                if score > best_score:
                    best_score = score
                    best_move = move

                # Update α if better value found
                if score > alpha:
                    alpha = score

            else:  # MIN NODE
                if score < best_score:
                    best_score = score
                    best_move = move

                # Update β if better value found
                if score < beta:
                    beta = score

            # STEP 4: PRUNING CONDITION - If α ≥ β, stop exploring
            if alpha >= beta:
                self.pruning_count += 1
                print(f"🌿 PRUNING at depth {depth}: α={alpha} ≥ β={beta}")

                # Store killer move (non-capture move that caused cutoff)
                if not board.is_capture(move):
                    self.store_killer_move(move, depth)

                # Update history table
                self.update_history(move, depth)

                break  # Prune remaining moves

        # Store in transposition table
        if best_move:
            if best_score <= original_alpha:
                node_type = NodeType.UPPER_BOUND
            elif best_score >= beta:
                node_type = NodeType.LOWER_BOUND
            else:
                node_type = NodeType.EXACT

            self.store_transposition(board, depth, best_score, node_type, best_move)

        return best_score, best_move

    def find_best_move(self, board: chess.Board) -> Tuple[chess.Move, Dict]:
        """
        Find the best move using Alpha-Beta pruning with iterative deepening
        """
        print("🧠 Starting Alpha-Beta Search...")
        print(f"📊 Position: {board.fen()}")

        start_time = time.time()
        self.nodes_searched = 0
        self.tt_hits = 0
        self.pruning_count = 0
        self.current_age += 1

        best_move = None
        best_score = 0

        # Iterative deepening for better move ordering
        for current_depth in range(1, self.max_depth + 1):
            print(f"\n🎯 Searching at depth {current_depth}")

            # STEP 1: Initialize α = -∞, β = +∞
            alpha = float('-inf')  # -∞
            beta = float('+inf')   # +∞

            print(f"   Step 1: Initialize α={alpha}, β={beta}")

            depth_start_time = time.time()

            # Start the alpha-beta search
            score, move = self.alpha_beta_search(board, current_depth, alpha, beta, board.turn)

            depth_time = time.time() - depth_start_time

            if move:
                best_move = move
                best_score = score

            print(f"   ⭐ Depth {current_depth}: Best move = {move}, Score = {score}")
            print(f"   ⏱️  Time: {depth_time:.3f}s, Nodes: {self.nodes_searched}")
            print(f"   🌿 Pruning events: {self.pruning_count}")

        total_time = time.time() - start_time

        # Statistics
        stats = {
            'nodes_searched': self.nodes_searched,
            'time_taken': total_time,
            'nodes_per_second': int(self.nodes_searched / total_time) if total_time > 0 else 0,
            'tt_hits': self.tt_hits,
            'pruning_count': self.pruning_count,
            'final_score': best_score
        }

        print(f"\n📈 SEARCH COMPLETE!")
        print(f"   🎯 Best Move: {best_move}")
        print(f"   📊 Final Score: {best_score}")
        print(f"   🔍 Nodes Searched: {self.nodes_searched:,}")
        print(f"   ⏱️  Total Time: {total_time:.3f}s")
        print(f"   🚀 NPS: {stats['nodes_per_second']:,}")
        print(f"   💾 TT Hits: {self.tt_hits}")
        print(f"   🌿 Prunings: {self.pruning_count}")

        return best_move, stats

def play_game():
    """Interactive game loop"""
    engine = AlphaBetaChessEngine(max_depth=4)  # Start with depth 4 for reasonable speed
    board = chess.Board()

    print("♟️  ALPHA-BETA CHESS ENGINE")
    print("=" * 50)
    print("Commands: 'move e2e4', 'quit', 'show', 'depth X'")
    print("=" * 50)

    while not board.is_game_over():
        print(f"\n{board}")
        print(f"Turn: {'White' if board.turn else 'Black'}")

        if board.turn:  # Human plays as White
            while True:
                try:
                    user_input = input("\nYour move: ").strip().lower()

                    if user_input == 'quit':
                        return
                    elif user_input == 'show':
                        print(f"FEN: {board.fen()}")
                        continue
                    elif user_input.startswith('depth '):
                        new_depth = int(user_input.split()[1])
                        engine.max_depth = new_depth
                        print(f"Search depth set to {new_depth}")
                        continue
                    elif user_input.startswith('move '):
                        move_str = user_input[5:]
                        move = chess.Move.from_uci(move_str)
                        if move in board.legal_moves:
                            board.push(move)
                            break
                        else:
                            print("Illegal move!")
                    else:
                        print("Invalid command!")
                except:
                    print("Invalid input! Try 'move e2e4'")

        else:  # AI plays as Black
            print("\n🤖 AI is thinking...")
            best_move, stats = engine.find_best_move(board)
            if best_move:
                print(f"AI plays: {best_move}")
                board.push(best_move)
            else:
                print("AI couldn't find a move!")
                break

    print(f"\n🎮 GAME OVER!")
    print(f"Result: {board.result()}")

    if board.is_checkmate():
        winner = "White" if not board.turn else "Black"
        print(f"🏆 {winner} wins by checkmate!")
    elif board.is_stalemate():
        print("🤝 Draw by stalemate!")
    elif board.is_insufficient_material():
        print("🤝 Draw by insufficient material!")

'''if __name__ == "__main__":
    print("🚀 Alpha-Beta Pruning Chess Engine v1.0")
    print("Features: Move Ordering, Transposition Tables, Quiescence Search")
    print("=" * 60)

    try:
        play_game()
    except KeyboardInterrupt:
        print("\n👋 Thanks for playing!")'''

🚀 Alpha-Beta Pruning Chess Engine v1.0
Features: Move Ordering, Transposition Tables, Quiescence Search
♟️  ALPHA-BETA CHESS ENGINE
Commands: 'move e2e4', 'quit', 'show', 'depth X'

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
Turn: White
Invalid input! Try 'move e2e4'
Invalid input! Try 'move e2e4'
Invalid input! Try 'move e2e4'
Invalid input! Try 'move e2e4'


In [ ]:
import chess
import chess.engine
import time
import math
import random
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass
import copy

@dataclass
class MCTSNode:
    """MCTS Node representing a game state"""
    board_state: str  # FEN string
    parent: Optional['MCTSNode']
    children: Dict[chess.Move, 'MCTSNode']
    move: Optional[chess.Move]  # Move that led to this node

    # MCTS Statistics
    visits: int = 0
    wins: float = 0.0  # Win score (can be fractional)

    # Node properties
    is_terminal: bool = False
    is_fully_expanded: bool = False
    untried_moves: List[chess.Move] = None

    def __post_init__(self):
        if self.untried_moves is None:
            self.untried_moves = []

    def ucb1_value(self, exploration_constant: float = math.sqrt(2)) -> float:
        """
        Calculate UCB1 (Upper Confidence Bound) value for node selection
        UCB1 = win_rate + C * sqrt(ln(parent_visits) / node_visits)
        """
        if self.visits == 0:
            return float('inf')  # Unvisited nodes have highest priority

        if self.parent is None or self.parent.visits == 0:
            return self.wins / self.visits

        exploitation = self.wins / self.visits
        exploration = exploration_constant * math.sqrt(math.log(self.parent.visits) / self.visits)

        return exploitation + exploration

    def add_child(self, move: chess.Move, board: chess.Board) -> 'MCTSNode':
        """Add a child node for the given move"""
        board_copy = board.copy()
        board_copy.push(move)

        child = MCTSNode(
            board_state=board_copy.fen(),
            parent=self,
            children={},
            move=move,
            is_terminal=board_copy.is_game_over()
        )

        self.children[move] = child
        return child

class MCTSChessEngine:
    def __init__(self, time_limit: float = 5.0, exploration_constant: float = math.sqrt(2)):
        self.time_limit = time_limit
        self.exploration_constant = exploration_constant

        # Statistics
        self.total_simulations = 0
        self.total_selections = 0
        self.total_expansions = 0
        self.total_backpropagations = 0

        # Evaluation function for rollout policy
        self.piece_values = {
            chess.PAWN: 1,
            chess.KNIGHT: 3,
            chess.BISHOP: 3,
            chess.ROOK: 5,
            chess.QUEEN: 9,
            chess.KING: 0
        }

    def evaluate_position_simple(self, board: chess.Board) -> float:
        """
        Simple evaluation function for rollout guidance
        Returns value from current player's perspective
        """
        if board.is_checkmate():
            return -1.0  # Current player loses
        if board.is_stalemate() or board.is_insufficient_material():
            return 0.0

        # Material count
        white_material = 0
        black_material = 0

        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is not None:
                value = self.piece_values[piece.piece_type]
                if piece.color == chess.WHITE:
                    white_material += value
                else:
                    black_material += value

        material_diff = white_material - black_material

        # Return from current player's perspective
        if board.turn == chess.WHITE:
            return material_diff / 50.0  # Normalize
        else:
            return -material_diff / 50.0

    def select_child(self, node: MCTSNode) -> MCTSNode:
        """
        PHASE 1: SELECTION
        Select child with highest UCB1 value
        """
        self.total_selections += 1

        print(f"🎯 PHASE 1 - SELECTION:")
        print(f"   Selecting from {len(node.children)} children")

        best_child = None
        best_value = -float('inf')

        for move, child in node.children.items():
            ucb1_value = child.ucb1_value(self.exploration_constant)

            print(f"   Move {move}: UCB1={ucb1_value:.3f} (visits={child.visits}, wins={child.wins:.2f})")

            if ucb1_value > best_value:
                best_value = ucb1_value
                best_child = child

        print(f"   ✅ Selected: {best_child.move} (UCB1={best_value:.3f})")
        return best_child

    def expand_node(self, node: MCTSNode, board: chess.Board) -> MCTSNode:
        """
        PHASE 2: EXPANSION
        Add one new child node to the tree
        """
        self.total_expansions += 1

        print(f"🌱 PHASE 2 - EXPANSION:")

        if node.is_terminal:
            print("   Node is terminal - no expansion needed")
            return node

        # Initialize untried moves if first expansion
        if not node.untried_moves and not node.is_fully_expanded:
            node.untried_moves = list(board.legal_moves)
            print(f"   Initialized {len(node.untried_moves)} untried moves")

        if not node.untried_moves:
            node.is_fully_expanded = True
            print("   Node is fully expanded")
            return node

        # Select random untried move
        move = random.choice(node.untried_moves)
        node.untried_moves.remove(move)

        # Create new child node
        child = node.add_child(move, board)

        print(f"   ✅ Expanded with move: {move}")
        print(f"   New child visits: {child.visits}, terminal: {child.is_terminal}")

        return child

    def simulate_game(self, board: chess.Board) -> float:
        """
        PHASE 3: SIMULATION
        Rollout policy inspired by Alpha-Beta pruning for non-volatile positions
        As mentioned in the text: "simulations follow a rollout policy inspired
        by Alpha-Beta pruning to find a non-volatile position"
        """
        self.total_simulations += 1

        print(f"🎲 PHASE 3 - SIMULATION:")
        print(f"   Starting rollout from position")

        simulation_board = board.copy()
        moves_played = 0
        max_moves = 100  # Prevent infinite games

        # Store initial player for final evaluation
        initial_player = simulation_board.turn

        while not simulation_board.is_game_over() and moves_played < max_moves:
            legal_moves = list(simulation_board.legal_moves)
            if not legal_moves:
                break

            # ROLLOUT POLICY inspired by Alpha-Beta pruning
            # Prefer moves that lead to non-volatile (quiet) positions
            move = self.select_rollout_move(simulation_board, legal_moves)
            simulation_board.push(move)
            moves_played += 1

        # Evaluate final position
        if simulation_board.is_checkmate():
            # If it's initial_player's turn and checkmate, initial_player loses
            result = -1.0 if simulation_board.turn == initial_player else 1.0
        elif simulation_board.is_stalemate() or simulation_board.is_insufficient_material():
            result = 0.0  # Draw
        else:
            # Use evaluation function from initial player's perspective
            result = self.evaluate_position_simple(simulation_board)
            if simulation_board.turn != initial_player:
                result = -result

        print(f"   🎯 Rollout complete: {moves_played} moves, result={result:.3f}")
        return result

    def select_rollout_move(self, board: chess.Board, legal_moves: List[chess.Move]) -> chess.Move:
        """
        Rollout policy inspired by Alpha-Beta pruning
        Seeks non-volatile positions by preferring:
        1. Captures that improve material balance
        2. Checks that maintain pressure
        3. Moves to quiet positions
        """
        scored_moves = []

        for move in legal_moves:
            score = 0

            # Prefer good captures
            if board.is_capture(move):
                captured_piece = board.piece_at(move.to_square)
                capturing_piece = board.piece_at(move.from_square)
                if captured_piece and capturing_piece:
                    # MVV-LVA style scoring
                    score += self.piece_values[captured_piece.piece_type] * 10
                    score -= self.piece_values[capturing_piece.piece_type]

            # Prefer checks (maintain pressure)
            if board.gives_check(move):
                score += 20

            # Prefer center control
            center_squares = [chess.E4, chess.E5, chess.D4, chess.D5]
            if move.to_square in center_squares:
                score += 5

            # Add some randomness for exploration
            score += random.uniform(-2, 2)

            scored_moves.append((score, move))

        # Select move with probability based on score
        scored_moves.sort(reverse=True)

        # Use weighted random selection (higher scores more likely)
        if len(scored_moves) > 3:
            # Pick from top 3 moves with weights [0.5, 0.3, 0.2]
            weights = [0.5, 0.3, 0.2]
            return random.choices([move for _, move in scored_moves[:3]], weights=weights)[0]
        else:
            # Pick from all moves with higher probability for better scores
            total_score = sum(max(0, score + 10) for score, _ in scored_moves)
            if total_score > 0:
                weights = [max(0, score + 10) / total_score for score, _ in scored_moves]
                return random.choices([move for _, move in scored_moves], weights=weights)[0]

        return random.choice(legal_moves)

    def backpropagate(self, node: MCTSNode, result: float):
        """
        PHASE 4: BACK-PROPAGATION
        Propagate the simulation result back up the tree
        Updates visit counts and win scores along the path back to root
        """
        self.total_backpropagations += 1

        print(f"⬆️  PHASE 4 - BACK-PROPAGATION:")
        print(f"   Propagating result: {result:.3f}")

        current_node = node
        path_length = 0

        while current_node is not None:
            # Update visit count
            current_node.visits += 1

            # Update win score - flip result for alternating players
            if path_length % 2 == 0:
                # Same player as simulation start
                current_node.wins += result
            else:
                # Opponent player
                current_node.wins -= result

            print(f"   Node {current_node.move if current_node.move else 'ROOT'}: "
                  f"visits={current_node.visits}, wins={current_node.wins:.2f}, "
                  f"win_rate={current_node.wins/current_node.visits:.3f}")

            current_node = current_node.parent
            path_length += 1

        print(f"   ✅ Backpropagated through {path_length} nodes")

    def mcts_search(self, board: chess.Board) -> Tuple[chess.Move, Dict]:
        """
        Main MCTS search loop:
        1. Selection - traverse tree using UCB1
        2. Expansion - add new node
        3. Simulation - rollout to terminal position
        4. Back-propagation - update statistics
        5. Repeat until time limit reached
        """
        print("🧠 Starting MCTS Search...")
        print(f"⏱️  Time limit: {self.time_limit} seconds")
        print("=" * 60)

        # Initialize root node
        root = MCTSNode(
            board_state=board.fen(),
            parent=None,
            children={},
            move=None
        )

        start_time = time.time()
        iterations = 0

        # Reset statistics
        self.total_simulations = 0
        self.total_selections = 0
        self.total_expansions = 0
        self.total_backpropagations = 0

        # Main MCTS loop
        while time.time() - start_time < self.time_limit:
            iterations += 1

            print(f"\n🔄 ITERATION {iterations}")
            print("-" * 40)

            # Start from root
            current_node = root
            current_board = board.copy()
            path = []

            # PHASE 1: SELECTION - traverse tree until leaf node
            while (current_node.children and
                   current_node.is_fully_expanded and
                   not current_node.is_terminal):

                current_node = self.select_child(current_node)
                current_board.push(current_node.move)
                path.append(current_node.move)

            # PHASE 2: EXPANSION - add new child if possible
            if not current_node.is_terminal and not current_node.is_fully_expanded:
                current_node = self.expand_node(current_node, current_board)
                if current_node.move:  # If we actually expanded
                    current_board.push(current_node.move)
                    path.append(current_node.move)

            # PHASE 3: SIMULATION - rollout from current position
            simulation_result = self.simulate_game(current_board)

            # PHASE 4: BACK-PROPAGATION - update tree statistics
            self.backpropagate(current_node, simulation_result)

            # Show progress every 10 iterations
            if iterations % 10 == 0:
                elapsed = time.time() - start_time
                print(f"\n📊 Progress: {iterations} iterations in {elapsed:.2f}s")

        total_time = time.time() - start_time

        # Select best move based on most visits (robust choice)
        if not root.children:
            # Fallback to random legal move
            legal_moves = list(board.legal_moves)
            return random.choice(legal_moves), {}

        best_move = max(root.children.keys(),
                       key=lambda move: root.children[move].visits)
        best_child = root.children[best_move]

        # Compile statistics
        stats = {
            'iterations': iterations,
            'time_taken': total_time,
            'iterations_per_second': iterations / total_time if total_time > 0 else 0,
            'total_selections': self.total_selections,
            'total_expansions': self.total_expansions,
            'total_simulations': self.total_simulations,
            'total_backpropagations': self.total_backpropagations,
            'best_move_visits': best_child.visits,
            'best_move_winrate': best_child.wins / best_child.visits if best_child.visits > 0 else 0,
            'tree_size': self.count_tree_nodes(root)
        }

        print(f"\n📈 MCTS SEARCH COMPLETE!")
        print("=" * 60)
        print(f"🎯 Best Move: {best_move}")
        print(f"📊 Move Statistics: {best_child.visits} visits, "
              f"{stats['best_move_winrate']:.3f} win rate")
        print(f"🔍 Total Iterations: {iterations}")
        print(f"⏱️  Total Time: {total_time:.3f}s")
        print(f"🚀 Iterations/sec: {stats['iterations_per_second']:.1f}")
        print(f"🌳 Tree Size: {stats['tree_size']} nodes")
        print(f"📈 Phase Counts:")
        print(f"   - Selections: {self.total_selections}")
        print(f"   - Expansions: {self.total_expansions}")
        print(f"   - Simulations: {self.total_simulations}")
        print(f"   - Back-propagations: {self.total_backpropagations}")

        # Show top 5 moves
        print(f"\n🏆 TOP MOVES:")
        sorted_moves = sorted(root.children.items(),
                            key=lambda x: x[1].visits, reverse=True)
        for i, (move, child) in enumerate(sorted_moves[:5]):
            win_rate = child.wins / child.visits if child.visits > 0 else 0
            print(f"   {i+1}. {move}: {child.visits} visits, "
                  f"{win_rate:.3f} win rate")

        return best_move, stats

    def count_tree_nodes(self, node: MCTSNode) -> int:
        """Count total nodes in the tree"""
        count = 1  # Count this node
        for child in node.children.values():
            count += self.count_tree_nodes(child)
        return count

def play_game():
    """Interactive game loop"""
    engine = MCTSChessEngine(time_limit=5.0)  # 5 seconds per move
    board = chess.Board()

    print("🎯 MONTE CARLO TREE SEARCH CHESS ENGINE")
    print("=" * 60)
    print("Commands: 'move e2e4', 'quit', 'show', 'time X'")
    print("=" * 60)

    while not board.is_game_over():
        print(f"\n{board}")
        print(f"Turn: {'White' if board.turn else 'Black'}")

        if board.turn:  # Human plays as White
            while True:
                try:
                    user_input = input("\nYour move: ").strip().lower()

                    if user_input == 'quit':
                        return
                    elif user_input == 'show':
                        print(f"FEN: {board.fen()}")
                        continue
                    elif user_input.startswith('time '):
                        new_time = float(user_input.split()[1])
                        engine.time_limit = new_time
                        print(f"Time limit set to {new_time} seconds")
                        continue
                    elif user_input.startswith('move '):
                        move_str = user_input[5:]
                        move = chess.Move.from_uci(move_str)
                        if move in board.legal_moves:
                            board.push(move)
                            break
                        else:
                            print("Illegal move!")
                    else:
                        print("Invalid command!")
                except:
                    print("Invalid input! Try 'move e2e4'")

        else:  # AI plays as Black
            print("\n🤖 MCTS AI is thinking...")
            best_move, stats = engine.mcts_search(board)
            print(f"AI plays: {best_move}")
            board.push(best_move)

    print(f"\n🎮 GAME OVER!")
    print(f"Result: {board.result()}")

    if board.is_checkmate():
        winner = "White" if not board.turn else "Black"
        print(f"🏆 {winner} wins by checkmate!")
    elif board.is_stalemate():
        print("🤝 Draw by stalemate!")
    elif board.is_insufficient_material():
        print("🤝 Draw by insufficient material!")

'''if __name__ == "__main__":
    print("🚀 Monte Carlo Tree Search Chess Engine v1.0")
    print("Features: UCB1 Selection, Smart Expansion, Alpha-Beta Inspired Rollouts")
    print("=" * 70)

    try:
        play_game()
    except KeyboardInterrupt:
        print("\n👋 Thanks for playing!")'''

🚀 Monte Carlo Tree Search Chess Engine v1.0
Features: UCB1 Selection, Smart Expansion, Alpha-Beta Inspired Rollouts
🎯 MONTE CARLO TREE SEARCH CHESS ENGINE
Commands: 'move e2e4', 'quit', 'show', 'time X'

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
Turn: White
Invalid input! Try 'move e2e4'

Your move: quit


In [ ]:
import chess
import chess.engine
import time
import math
import random
import json
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, asdict
from enum import Enum
from datetime import datetime
import numpy as np
from pathlib import Path
import threading
import concurrent.futures

class EvaluationLevel(Enum):
    BASIC = "basic"           # Material only
    MEDIUM = "medium"         # Material + positional
    ADVANCED = "advanced"     # All features + endgame

class TestType(Enum):
    EXPLORATION_BIAS = "exploration_bias"
    DIFFERENT_THINK_TIMES = "different_think_times"
    ASYMMETRIC_TIME = "asymmetric_time"
    EVALUATION_COMPLEXITY = "evaluation_complexity"
    HEAD_TO_HEAD = "head_to_head"

@dataclass
class GameResult:
    game_id: str
    white_engine: str
    black_engine: str
    white_config: Dict[str, Any]
    black_config: Dict[str, Any]
    result: str  # "1-0", "0-1", "1/2-1/2"
    moves_count: int
    time_taken: float
    starting_position: str
    termination_reason: str
    white_stats: Dict[str, Any]
    black_stats: Dict[str, Any]
    timestamp: str

@dataclass
class TournamentStats:
    total_games: int
    white_wins: int
    black_wins: int
    draws: int
    avg_moves: float
    avg_time: float
    win_rate_white: float
    win_rate_black: float
    draw_rate: float

class EvaluationFunctions:
    """Three levels of evaluation complexity as described in the research"""

    def __init__(self):
        self.piece_values = {
            chess.PAWN: 100,
            chess.KNIGHT: 320,
            chess.BISHOP: 330,
            chess.ROOK: 500,
            chess.QUEEN: 900,
            chess.KING: 20000
        }

        # Piece-Square Tables (from research document)
        self.pawn_pst = [
            [  0,  0,  0,  0,  0,  0,  0,  0],
            [ 50, 50, 50, 50, 50, 50, 50, 50],
            [ 10, 10, 20, 30, 30, 20, 10, 10],
            [  5,  5, 10, 25, 25, 10,  5,  5],
            [  0,  0,  0, 20, 20,  0,  0,  0],
            [  5, -5,-10,  0,  0,-10, -5,  5],
            [  5, 10, 10,-20,-20, 10, 10,  5],
            [  0,  0,  0,  0,  0,  0,  0,  0]
        ]

        self.knight_pst = [
            [-50,-40,-30,-30,-30,-30,-40,-50],
            [-40,-20,  0,  0,  0,  0,-20,-40],
            [-30,  0, 10, 15, 15, 10,  0,-30],
            [-30,  5, 15, 20, 20, 15,  5,-30],
            [-30,  0, 15, 20, 20, 15,  0,-30],
            [-30,  5, 10, 15, 15, 10,  5,-30],
            [-40,-20,  0,  5,  5,  0,-20,-40],
            [-50,-40,-30,-30,-30,-30,-40,-50]
        ]

        self.bishop_pst = [
            [-20,-10,-10,-10,-10,-10,-10,-20],
            [-10,  0,  0,  0,  0,  0,  0,-10],
            [-10,  0,  5, 10, 10,  5,  0,-10],
            [-10,  5,  5, 10, 10,  5,  5,-10],
            [-10,  0, 10, 10, 10, 10,  0,-10],
            [-10, 10, 10, 10, 10, 10, 10,-10],
            [-10,  5,  0,  0,  0,  0,  5,-10],
            [-20,-10,-10,-10,-10,-10,-10,-20]
        ]

    def basic_evaluation(self, board: chess.Board) -> int:
        """BASIC: Material count only"""
        if board.is_checkmate():
            return -20000 if board.turn else 20000
        if board.is_stalemate() or board.is_insufficient_material():
            return 0

        score = 0
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is None:
                continue

            piece_value = self.piece_values[piece.piece_type]
            if piece.color:  # White
                score += piece_value
            else:  # Black
                score -= piece_value

        return score

    def medium_evaluation(self, board: chess.Board) -> int:
        """MEDIUM: Material + Positional (PST + Mobility)"""
        score = self.basic_evaluation(board)

        if abs(score) >= 20000:  # Game over
            return score

        # Add positional scores
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is None:
                continue

            row, col = divmod(square, 8)
            if not piece.color:  # Black pieces
                row = 7 - row

            positional_bonus = 0
            if piece.piece_type == chess.PAWN:
                positional_bonus = self.pawn_pst[row][col]
            elif piece.piece_type == chess.KNIGHT:
                positional_bonus = self.knight_pst[row][col]
            elif piece.piece_type == chess.BISHOP:
                positional_bonus = self.bishop_pst[row][col]

            if piece.color:  # White
                score += positional_bonus
            else:  # Black
                score -= positional_bonus

        # Add mobility score for Bishop, Rook, Knight (as per research)
        score += self._calculate_mobility_score(board)

        return score

    def advanced_evaluation(self, board: chess.Board) -> int:
        """ADVANCED: All features + endgame considerations"""
        score = self.medium_evaluation(board)

        if abs(score) >= 20000:  # Game over
            return score

        # Calculate endgame weight
        endgame_weight = self._calculate_endgame_weight(board)

        # Castling penalty (ignored in endgame)
        if endgame_weight < 0.8:
            score += self._calculate_castling_penalty(board)

        # Endgame checkmate assistance
        if endgame_weight > 0.6:
            score += self._calculate_endgame_assistance(board) * endgame_weight

        return score

    def _calculate_mobility_score(self, board: chess.Board) -> int:
        """Calculate mobility score using Sm = 10√n formula"""
        white_mobility = 0
        black_mobility = 0

        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is None:
                continue

            # Only calculate for Bishop, Rook, Knight
            if piece.piece_type not in [chess.BISHOP, chess.ROOK, chess.KNIGHT]:
                continue

            # Count legal moves from this square
            moves_count = 0
            for move in board.legal_moves:
                if move.from_square == square:
                    moves_count += 1

            mobility_score = int(10 * math.sqrt(moves_count)) if moves_count > 0 else 0

            if piece.color:  # White
                white_mobility += mobility_score
            else:  # Black
                black_mobility += mobility_score

        return white_mobility - black_mobility

    def _calculate_endgame_weight(self, board: chess.Board) -> float:
        """Calculate endgame weight based on remaining material"""
        total_material = 0
        starting_material = 8*100 + 4*320 + 4*330 + 4*500 + 2*900  # Starting material

        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece and piece.piece_type != chess.KING:
                total_material += self.piece_values[piece.piece_type]

        return 1.0 - (total_material / starting_material)

    def _calculate_castling_penalty(self, board: chess.Board) -> int:
        """Penalty for not castling, increases with opponent development"""
        penalty = 0

        # White castling penalty
        if board.has_kingside_castling_rights(chess.WHITE) or board.has_queenside_castling_rights(chess.WHITE):
            # Calculate opponent development score (normalized)
            opponent_development = self._calculate_development_score(board, chess.BLACK)
            penalty -= int(opponent_development * 50)  # Up to 50 centipawn penalty

        # Black castling penalty
        if board.has_kingside_castling_rights(chess.BLACK) or board.has_queenside_castling_rights(chess.BLACK):
            opponent_development = self._calculate_development_score(board, chess.WHITE)
            penalty += int(opponent_development * 50)

        return penalty

    def _calculate_development_score(self, board: chess.Board, color: chess.Color) -> float:
        """Calculate development score [0,1] based on piece-square evaluation"""
        development_score = 0
        max_development = 0

        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece and piece.color == color:
                row, col = divmod(square, 8)
                if not color:  # Black pieces
                    row = 7 - row

                if piece.piece_type == chess.KNIGHT:
                    development_score += max(0, self.knight_pst[row][col])
                    max_development += 20  # Max knight development bonus
                elif piece.piece_type == chess.BISHOP:
                    development_score += max(0, self.bishop_pst[row][col])
                    max_development += 10  # Max bishop development bonus

        return development_score / max_development if max_development > 0 else 0

    def _calculate_endgame_assistance(self, board: chess.Board) -> int:
        """Endgame checkmate assistance - force king to corners, bring own king closer"""
        score = 0

        white_king_sq = board.king(chess.WHITE)
        black_king_sq = board.king(chess.BLACK)

        if white_king_sq is None or black_king_sq is None:
            return 0

        # Force opponent king to corners
        black_king_row, black_king_col = divmod(black_king_sq, 8)
        corner_distance = min(
            abs(black_king_row - 0) + abs(black_king_col - 0),  # a8
            abs(black_king_row - 0) + abs(black_king_col - 7),  # h8
            abs(black_king_row - 7) + abs(black_king_col - 0),  # a1
            abs(black_king_row - 7) + abs(black_king_col - 7)   # h1
        )
        score += (14 - corner_distance) * 10  # Reward forcing to corners

        # Bring own king closer to opponent king
        white_king_row, white_king_col = divmod(white_king_sq, 8)
        king_distance = abs(white_king_row - black_king_row) + abs(white_king_col - black_king_col)
        score += (14 - king_distance) * 5  # Reward king proximity

        return score

class ConfigurableEngine:
    """Wrapper class to make engines configurable with different evaluation levels"""

    def __init__(self, engine_type: str, evaluation_level: EvaluationLevel, **kwargs):
        self.engine_type = engine_type
        self.evaluation_level = evaluation_level
        self.config = kwargs
        self.evaluator = EvaluationFunctions()

        # Import engines from your existing code
        if engine_type == "MCTS":
            # Assuming your MCTS code is in mcts_engine.py
            self.engine = MCTSChessEngine(**kwargs)
        elif engine_type == "ABP":
            # Assuming your ABP code is in alpha_beta_engine.py
            self.engine = AlphaBetaChessEngine(**kwargs)

        # Override the evaluation function
        self._set_evaluation_function()

    def _set_evaluation_function(self):
        """Set the appropriate evaluation function based on level"""
        if self.evaluation_level == EvaluationLevel.BASIC:
            eval_func = self.evaluator.basic_evaluation
        elif self.evaluation_level == EvaluationLevel.MEDIUM:
            eval_func = self.evaluator.medium_evaluation
        else:  # ADVANCED
            eval_func = self.evaluator.advanced_evaluation

        # Replace the engine's evaluation function
        if self.engine_type == "MCTS":
            self.engine.evaluate_position_simple = eval_func
        elif self.engine_type == "ABP":
            self.engine.evaluate_position = eval_func

    def get_move(self, board: chess.Board) -> Tuple[chess.Move, Dict]:
        """Get the best move from the engine"""
        if self.engine_type == "MCTS":
            return self.engine.mcts_search(board)
        elif self.engine_type == "ABP":
            return self.engine.find_best_move(board)

class ChessTournament:
    """Main tournament system for running comprehensive tests"""

    def __init__(self, results_dir: str = "tournament_results"):
        self.results_dir = Path(results_dir)
        self.results_dir.mkdir(exist_ok=True)
        self.games_played = []

        # Load starting positions (generate random ones if file doesn't exist)
        self.starting_positions = self._load_starting_positions()

    def _load_starting_positions(self) -> List[str]:
        """Load starting positions or generate random ones"""
        positions_file = self.results_dir / "starting_positions.txt"

        if positions_file.exists():
            with open(positions_file, 'r') as f:
                return [line.strip() for line in f if line.strip()]
        else:
            # Generate 500 random positions
            print("Generating 500 random starting positions...")
            positions = []
            for _ in range(500):
                board = chess.Board()
                # Play 10-15 random moves to get varied positions
                for _ in range(random.randint(10, 15)):
                    if board.is_game_over():
                        break
                    moves = list(board.legal_moves)
                    if moves:
                        board.push(random.choice(moves))

                positions.append(board.fen())

            # Save positions
            with open(positions_file, 'w') as f:
                for pos in positions:
                    f.write(pos + '\n')

            return positions

    def play_game(self, white_engine: ConfigurableEngine, black_engine: ConfigurableEngine,
                  starting_position: str = None, game_id: str = None) -> GameResult:
        """Play a single game between two engines"""

        board = chess.Board(starting_position) if starting_position else chess.Board()
        moves_count = 0
        start_time = time.time()

        white_stats = {}
        black_stats = {}

        print(f"🎯 Playing game {game_id}: {white_engine.engine_type} vs {black_engine.engine_type}")

        while not board.is_game_over() and moves_count < 200:  # Max 200 moves
            current_engine = white_engine if board.turn else black_engine

            try:
                move, stats = current_engine.get_move(board)
                board.push(move)
                moves_count += 1

                # Store stats
                if board.turn:  # Just moved, so it was the other player
                    black_stats = stats
                else:
                    white_stats = stats

            except Exception as e:
                print(f"Engine error: {e}")
                break

        # Determine result
        result = board.result()
        total_time = time.time() - start_time

        # Determine termination reason
        if board.is_checkmate():
            termination = "checkmate"
        elif board.is_stalemate():
            termination = "stalemate"
        elif board.is_insufficient_material():
            termination = "insufficient_material"
        elif moves_count >= 200:
            termination = "move_limit"
            result = "1/2-1/2"  # Draw by move limit
        else:
            termination = "unknown"

        game_result = GameResult(
            game_id=game_id or f"game_{len(self.games_played)}",
            white_engine=white_engine.engine_type,
            black_engine=black_engine.engine_type,
            white_config=white_engine.config,
            black_config=black_engine.config,
            result=result,
            moves_count=moves_count,
            time_taken=total_time,
            starting_position=starting_position or board.starting_fen,
            termination_reason=termination,
            white_stats=white_stats,
            black_stats=black_stats,
            timestamp=datetime.now().isoformat()
        )

        self.games_played.append(game_result)
        print(f"✅ Game {game_id} finished: {result} ({termination}) in {moves_count} moves")

        return game_result

    def run_exploration_bias_test(self, base_c: float = 1.0, test_values: List[float] = None,
                                  games_per_test: int = 50) -> Dict[str, Any]:
        """Test 1: MCTS Exploration Bias Testing"""
        print("🔬 RUNNING TEST 1: MCTS Exploration Bias")
        print("=" * 60)

        if test_values is None:
            test_values = [0.5, 1.0, 1.4, 2.0, 2.8]

        results = {}
        base_engine = ConfigurableEngine("MCTS", EvaluationLevel.MEDIUM,
                                       time_limit=1.0, exploration_constant=base_c)

        for c_value in test_values:
            print(f"\n🎯 Testing C = {c_value} vs C = {base_c}")

            test_engine = ConfigurableEngine("MCTS", EvaluationLevel.MEDIUM,
                                           time_limit=1.0, exploration_constant=c_value)

            game_results = []
            for i in range(games_per_test):
                position = random.choice(self.starting_positions)

                # Play both colors
                result1 = self.play_game(test_engine, base_engine, position,
                                       f"exp_bias_c{c_value}_g{i}_w")
                result2 = self.play_game(base_engine, test_engine, position,
                                       f"exp_bias_c{c_value}_g{i}_b")

                game_results.extend([result1, result2])

            # Calculate statistics
            stats = self._calculate_stats(game_results, test_engine.engine_type)
            results[c_value] = stats

            print(f"   Results: {stats.win_rate_white:.3f} win rate")

        # Save results
        self._save_test_results("exploration_bias", results)
        return results

    def run_think_time_test(self, think_times: List[float] = None, games_per_time: int = 100) -> Dict[str, Any]:
        """Test 2: Performance at Different Think Times"""
        print("🔬 RUNNING TEST 2: Performance at Different Think Times")
        print("=" * 60)

        if think_times is None:
            think_times = [0.1, 0.25, 0.5, 1.0, 2.0]

        results = {}

        for think_time in think_times:
            print(f"\n🎯 Testing at {think_time}s think time")

            mcts_engine = ConfigurableEngine("MCTS", EvaluationLevel.MEDIUM, time_limit=think_time)
            abp_engine = ConfigurableEngine("ABP", EvaluationLevel.MEDIUM, time_limit=think_time)

            game_results = []
            for i in range(games_per_time // 2):  # Divide by 2 since we play both colors
                position = random.choice(self.starting_positions)

                result1 = self.play_game(mcts_engine, abp_engine, position,
                                       f"think_time_{think_time}_g{i}_mw")
                result2 = self.play_game(abp_engine, mcts_engine, position,
                                       f"think_time_{think_time}_g{i}_mb")

                game_results.extend([result1, result2])

            stats = self._calculate_stats(game_results, "MCTS")
            results[think_time] = stats

            print(f"   MCTS vs ABP: {stats.win_rate_white:.3f} win rate")

        self._save_test_results("think_time", results)
        return results

    def run_asymmetric_time_test(self, abp_time: float = 0.1, max_mcts_time: float = 5.0,
                                step: float = 0.5, games_per_test: int = 100) -> Dict[str, Any]:
        """Test 3: MCTS with more time vs ABP with fixed time"""
        print("🔬 RUNNING TEST 3: Asymmetric Time Testing")
        print("=" * 60)

        results = {}
        abp_engine = ConfigurableEngine("ABP", EvaluationLevel.MEDIUM, time_limit=abp_time)

        mcts_time = 0.1
        while mcts_time <= max_mcts_time:
            print(f"\n🎯 Testing MCTS({mcts_time}s) vs ABP({abp_time}s)")

            mcts_engine = ConfigurableEngine("MCTS", EvaluationLevel.MEDIUM, time_limit=mcts_time)

            game_results = []
            for i in range(games_per_test // 2):
                position = random.choice(self.starting_positions)

                result1 = self.play_game(mcts_engine, abp_engine, position,
                                       f"asym_time_m{mcts_time}_g{i}_mw")
                result2 = self.play_game(abp_engine, mcts_engine, position,
                                       f"asym_time_m{mcts_time}_g{i}_mb")

                game_results.extend([result1, result2])

            stats = self._calculate_stats(game_results, "MCTS")
            results[mcts_time] = stats

            print(f"   MCTS win rate: {stats.win_rate_white:.3f}")

            # Stop if MCTS clearly outperforms ABP
            if stats.win_rate_white > 0.55:
                print("   🎉 MCTS achieved superior performance!")
                break

            mcts_time += step

        self._save_test_results("asymmetric_time", results)
        return results

    def run_evaluation_complexity_test(self, games_per_level: int = 250) -> Dict[str, Any]:
        """Test 4: Different Evaluation Complexity Levels"""
        print("🔬 RUNNING TEST 4: Evaluation Complexity Testing")
        print("=" * 60)

        results = {}
        levels = [EvaluationLevel.BASIC, EvaluationLevel.MEDIUM, EvaluationLevel.ADVANCED]

        for level in levels:
            print(f"\n🎯 Testing at {level.value} complexity")

            mcts_engine = ConfigurableEngine("MCTS", level, time_limit=1.0)
            abp_engine = ConfigurableEngine("ABP", level, time_limit=1.0)

            game_results = []
            for i in range(games_per_level // 2):
                position = random.choice(self.starting_positions)

                result1 = self.play_game(mcts_engine, abp_engine, position,
                                       f"eval_{level.value}_g{i}_mw")
                result2 = self.play_game(abp_engine, mcts_engine, position,
                                       f"eval_{level.value}_g{i}_mb")

                game_results.extend([result1, result2])

            stats = self._calculate_stats(game_results, "MCTS")
            results[level.value] = stats

            print(f"   MCTS vs ABP: {stats.win_rate_white:.3f} win rate")

        self._save_test_results("evaluation_complexity", results)
        return results

    def run_head_to_head_tournament(self, games: int = 250) -> Dict[str, Any]:
        """Test 5: Full Head-to-Head Tournament"""
        print("🔬 RUNNING TEST 5: Head-to-Head Tournament")
        print("=" * 60)

        mcts_engine = ConfigurableEngine("MCTS", EvaluationLevel.ADVANCED, time_limit=2.0)
        abp_engine = ConfigurableEngine("ABP", EvaluationLevel.ADVANCED, time_limit=2.0)

        game_results = []
        positions_used = self.starting_positions[:games//2]  # Use first half of positions

        for i, position in enumerate(positions_used):
            # Play each position twice (both colors)
            result1 = self.play_game(mcts_engine, abp_engine, position, f"tournament_g{i}_mw")
            result2 = self.play_game(abp_engine, mcts_engine, position, f"tournament_g{i}_mb")

            game_results.extend([result1, result2])

            if (i + 1) % 25 == 0:
                print(f"   Progress: {i+1}/{len(positions_used)} positions completed")

        stats = self._calculate_stats(game_results, "MCTS")
        results = {"tournament": stats}

        print(f"\n🏆 TOURNAMENT COMPLETE!")
        print(f"   MCTS vs ABP: {stats.win_rate_white:.3f} win rate")
        print(f"   Total games: {len(game_results)}")
        print(f"   Draws: {stats.draw_rate:.3f}")

        self._save_test_results("head_to_head", results)
        return results

    def _calculate_stats(self, game_results: List[GameResult], focus_engine: str) -> TournamentStats:
        """Calculate tournament statistics"""
        total_games = len(game_results)
        focus_wins = 0
        opponent_wins = 0
        draws = 0
        total_moves = 0
        total_time = 0.0

        for game in game_results:
            total_moves += game.moves_count
            total_time += game.time_taken

            # Determine if focus_engine won, lost, or drew
            if game.result == "1/2-1/2":
                draws += 1
            elif (game.result == "1-0" and game.white_engine == focus_engine) or \
                 (game.result == "0-1" and game.black_engine == focus_engine):
                focus_wins += 1
            else:
                opponent_wins += 1

        return TournamentStats(
            total_games=total_games,
            white_wins=focus_wins,
            black_wins=opponent_wins,
            draws=draws,
            avg_moves=total_moves / total_games if total_games > 0 else 0,
            avg_time=total_time / total_games if total_games > 0 else 0,
            win_rate_white=focus_wins / total_games if total_games > 0 else 0,
            win_rate_black=opponent_wins / total_games if total_games > 0 else 0,
            draw_rate=draws / total_games if total_games > 0 else 0
        )

    def _save_test_results(self, test_name: str, results: Dict[str, Any]):
        """Save test results to JSON file"""
        filename = self.results_dir / f"{test_name}_results.json"

        # Convert dataclass objects to dictionaries
        serializable_results = {}
        for key, value in results.items():
            if hasattr(value, '__dict__'):
                serializable_results[key] = asdict(value)
            else:
                serializable_results[key] = value

        with open(filename, 'w') as f:
            json.dump(serializable_results, f, indent=2)

        print(f"💾 Results saved to {filename}")

    def generate_report(self):
        """Generate comprehensive analysis report"""
        print("📊 GENERATING COMPREHENSIVE REPORT")
        print("=" * 60)

        report = {
            "tournament_summary": {
                "total_games": len(self.games_played),
                "timestamp": datetime.now().isoformat(),
                "test_types_completed": []
            }
        }

        # Analyze results from each test
        for result_file in self.results_dir.glob("*_results.json"):
            test_name = result_file.stem.replace("_results", "")
            with open(result_file, 'r') as f:
                test_results = json.load(f)

            report["tournament_summary"]["test_types_completed"].append(test_name)
            report[test_name] = test_results

        # Save comprehensive report
        report_file = self.results_dir / "comprehensive_report.json"
        with open(report_file, 'w') as f:
            json.dump(report, f, indent=2)

        # Generate CSV summary for easy analysis
        self._generate_csv_summary()

        # Generate visualizations
        self._generate_visualizations()

        print(f"📈 Comprehensive report saved to {report_file}")

    def _generate_csv_summary(self):
        """Generate CSV summary of all games"""
        csv_file = self.results_dir / "all_games_summary.csv"

        with open(csv_file, 'w', newline='') as f:
            if not self.games_played:
                return

            fieldnames = list(asdict(self.games_played[0]).keys())
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()

            for game in self.games_played:
                # Convert complex objects to strings for CSV
                game_dict = asdict(game)
                for key, value in game_dict.items():
                    if isinstance(value, dict):
                        game_dict[key] = json.dumps(value)
                writer.writerow(game_dict)

        print(f"📊 CSV summary saved to {csv_file}")

    def _generate_visualizations(self):
        """Generate visualization plots"""
        try:
            import matplotlib.pyplot as plt
            import seaborn as sns

            plt.style.use('seaborn-v0_8')
            fig, axes = plt.subplots(2, 2, figsize=(15, 12))
            fig.suptitle('Chess Engine Testing Results', fontsize=16, fontweight='bold')

            # Plot 1: Win rates by test type
            self._plot_win_rates(axes[0, 0])

            # Plot 2: Performance vs think time
            self._plot_think_time_performance(axes[0, 1])

            # Plot 3: Evaluation complexity comparison
            self._plot_evaluation_complexity(axes[1, 0])

            # Plot 4: Game length distribution
            self._plot_game_length_distribution(axes[1, 1])

            plt.tight_layout()
            plot_file = self.results_dir / "analysis_plots.png"
            plt.savefig(plot_file, dpi=300, bbox_inches='tight')
            plt.close()

            print(f"📈 Visualizations saved to {plot_file}")

        except ImportError:
            print("⚠️  matplotlib/seaborn not available - skipping visualizations")
        except Exception as e:
            print(f"⚠️  Error generating visualizations: {e}")

    def _plot_win_rates(self, ax):
        """Plot win rates by test type"""
        test_files = list(self.results_dir.glob("*_results.json"))
        if not test_files:
            ax.text(0.5, 0.5, 'No test results available', ha='center', va='center')
            ax.set_title('Win Rates by Test Type')
            return

        test_names = []
        win_rates = []

        for result_file in test_files:
            test_name = result_file.stem.replace("_results", "")
            with open(result_file, 'r') as f:
                results = json.load(f)

            # Extract average win rate from results
            if results:
                avg_win_rate = 0
                count = 0
                for key, value in results.items():
                    if isinstance(value, dict) and 'win_rate_white' in value:
                        avg_win_rate += value['win_rate_white']
                        count += 1

                if count > 0:
                    test_names.append(test_name.replace('_', ' ').title())
                    win_rates.append(avg_win_rate / count)

        if test_names:
            bars = ax.bar(test_names, win_rates, color='steelblue', alpha=0.7)
            ax.set_ylabel('MCTS Win Rate')
            ax.set_title('MCTS Win Rate by Test Type')
            ax.set_ylim(0, 1)
            plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

            # Add value labels on bars
            for bar, rate in zip(bars, win_rates):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{rate:.3f}', ha='center', va='bottom')

    def _plot_think_time_performance(self, ax):
        """Plot performance vs think time"""
        think_time_file = self.results_dir / "think_time_results.json"
        if not think_time_file.exists():
            ax.text(0.5, 0.5, 'Think time results not available', ha='center', va='center')
            ax.set_title('Performance vs Think Time')
            return

        with open(think_time_file, 'r') as f:
            results = json.load(f)

        times = []
        win_rates = []

        for time_str, stats in results.items():
            times.append(float(time_str))
            win_rates.append(stats['win_rate_white'])

        if times:
            # Sort by time
            sorted_data = sorted(zip(times, win_rates))
            times, win_rates = zip(*sorted_data)

            ax.plot(times, win_rates, marker='o', linewidth=2, markersize=8, color='darkgreen')
            ax.set_xlabel('Think Time (seconds)')
            ax.set_ylabel('MCTS Win Rate')
            ax.set_title('MCTS Performance vs Think Time')
            ax.grid(True, alpha=0.3)
            ax.set_ylim(0, 1)

    def _plot_evaluation_complexity(self, ax):
        """Plot evaluation complexity comparison"""
        eval_file = self.results_dir / "evaluation_complexity_results.json"
        if not eval_file.exists():
            ax.text(0.5, 0.5, 'Evaluation complexity results not available', ha='center', va='center')
            ax.set_title('Evaluation Complexity Comparison')
            return

        with open(eval_file, 'r') as f:
            results = json.load(f)

        levels = []
        win_rates = []
        draw_rates = []

        level_order = ['basic', 'medium', 'advanced']
        for level in level_order:
            if level in results:
                levels.append(level.title())
                win_rates.append(results[level]['win_rate_white'])
                draw_rates.append(results[level]['draw_rate'])

        if levels:
            x = np.arange(len(levels))
            width = 0.35

            bars1 = ax.bar(x - width/2, win_rates, width, label='MCTS Win Rate', color='steelblue', alpha=0.7)
            bars2 = ax.bar(x + width/2, draw_rates, width, label='Draw Rate', color='orange', alpha=0.7)

            ax.set_xlabel('Evaluation Complexity')
            ax.set_ylabel('Rate')
            ax.set_title('Performance by Evaluation Complexity')
            ax.set_xticks(x)
            ax.set_xticklabels(levels)
            ax.legend()
            ax.set_ylim(0, 1)

            # Add value labels
            for bars in [bars1, bars2]:
                for bar in bars:
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                           f'{height:.3f}', ha='center', va='bottom', fontsize=8)

    def _plot_game_length_distribution(self, ax):
        """Plot game length distribution"""
        if not self.games_played:
            ax.text(0.5, 0.5, 'No games data available', ha='center', va='center')
            ax.set_title('Game Length Distribution')
            return

        moves_counts = [game.moves_count for game in self.games_played]

        ax.hist(moves_counts, bins=20, color='purple', alpha=0.7, edgecolor='black')
        ax.set_xlabel('Number of Moves')
        ax.set_ylabel('Frequency')
        ax.set_title('Game Length Distribution')
        ax.axvline(np.mean(moves_counts), color='red', linestyle='--',
                  label=f'Mean: {np.mean(moves_counts):.1f}')
        ax.legend()

# Main execution class
class TournamentRunner:
    """Main class to run the complete testing suite"""

    def __init__(self):
        self.tournament = ChessTournament()

    def run_all_tests(self, quick_mode: bool = False):
        """Run all tests in the research methodology"""
        print("🚀 STARTING COMPREHENSIVE CHESS ENGINE TESTING SUITE")
        print("=" * 70)
        print(f"Quick mode: {'ON' if quick_mode else 'OFF'}")
        print("=" * 70)

        start_time = time.time()

        if quick_mode:
            # Reduced games for faster testing
            games_config = {
                'exploration_games': 20,
                'think_time_games': 40,
                'asymmetric_games': 40,
                'complexity_games': 60,
                'tournament_games': 100
            }
        else:
            # Full research configuration
            games_config = {
                'exploration_games': 50,
                'think_time_games': 100,
                'asymmetric_games': 100,
                'complexity_games': 250,
                'tournament_games': 250
            }

        try:
            # Test 1: MCTS Exploration Bias
            print("\n" + "="*50)
            exploration_results = self.tournament.run_exploration_bias_test(
                games_per_test=games_config['exploration_games']
            )

            # Test 2: Think Time Performance
            print("\n" + "="*50)
            think_time_results = self.tournament.run_think_time_test(
                games_per_time=games_config['think_time_games']
            )

            # Test 3: Asymmetric Time Testing
            print("\n" + "="*50)
            asymmetric_results = self.tournament.run_asymmetric_time_test(
                games_per_test=games_config['asymmetric_games']
            )

            # Test 4: Evaluation Complexity
            print("\n" + "="*50)
            complexity_results = self.tournament.run_evaluation_complexity_test(
                games_per_level=games_config['complexity_games']
            )

            # Test 5: Head-to-Head Tournament
            print("\n" + "="*50)
            tournament_results = self.tournament.run_head_to_head_tournament(
                games=games_config['tournament_games']
            )

            # Generate comprehensive report
            print("\n" + "="*50)
            self.tournament.generate_report()

            total_time = time.time() - start_time
            total_games = len(self.tournament.games_played)

            print(f"\n🎉 ALL TESTS COMPLETED!")
            print(f"   Total time: {total_time/3600:.2f} hours")
            print(f"   Total games: {total_games}")
            print(f"   Games per hour: {total_games/(total_time/3600):.1f}")
            print(f"   Results saved to: {self.tournament.results_dir}")

            return {
                'exploration': exploration_results,
                'think_time': think_time_results,
                'asymmetric': asymmetric_results,
                'complexity': complexity_results,
                'tournament': tournament_results
            }

        except KeyboardInterrupt:
            print(f"\n⚠️  Testing interrupted by user")
            print(f"   Games completed so far: {len(self.tournament.games_played)}")
            self.tournament.generate_report()
            return None

        except Exception as e:
            print(f"\n❌ Error during testing: {e}")
            print(f"   Games completed so far: {len(self.tournament.games_played)}")
            self.tournament.generate_report()
            raise

    def run_single_test(self, test_type: TestType, **kwargs):
        """Run a single specific test"""
        print(f"🎯 RUNNING SINGLE TEST: {test_type.value}")
        print("=" * 50)

        if test_type == TestType.EXPLORATION_BIAS:
            return self.tournament.run_exploration_bias_test(**kwargs)
        elif test_type == TestType.DIFFERENT_THINK_TIMES:
            return self.tournament.run_think_time_test(**kwargs)
        elif test_type == TestType.ASYMMETRIC_TIME:
            return self.tournament.run_asymmetric_time_test(**kwargs)
        elif test_type == TestType.EVALUATION_COMPLEXITY:
            return self.tournament.run_evaluation_complexity_test(**kwargs)
        elif test_type == TestType.HEAD_TO_HEAD:
            return self.tournament.run_head_to_head_tournament(**kwargs)

    def run_custom_matchup(self, engine1_config: Dict, engine2_config: Dict, games: int = 50):
        """Run a custom matchup between two engine configurations"""
        print("🎯 RUNNING CUSTOM MATCHUP")
        print("=" * 40)

        engine1 = ConfigurableEngine(**engine1_config)
        engine2 = ConfigurableEngine(**engine2_config)

        game_results = []
        for i in range(games // 2):
            position = random.choice(self.tournament.starting_positions)

            result1 = self.tournament.play_game(engine1, engine2, position, f"custom_g{i}_1w")
            result2 = self.tournament.play_game(engine2, engine1, position, f"custom_g{i}_2w")

            game_results.extend([result1, result2])

        stats = self.tournament._calculate_stats(game_results, engine1_config['engine_type'])

        print(f"\n📊 CUSTOM MATCHUP RESULTS:")
        print(f"   {engine1_config['engine_type']} vs {engine2_config['engine_type']}")
        print(f"   Win rate: {stats.win_rate_white:.3f}")
        print(f"   Draw rate: {stats.draw_rate:.3f}")
        print(f"   Average moves: {stats.avg_moves:.1f}")

        return stats

# Example usage and testing functions
def demo_quick_test():
    """Run a quick demo test to verify everything works"""
    print("🚀 RUNNING QUICK DEMO TEST")
    print("=" * 40)

    runner = TournamentRunner()

    # Quick exploration bias test
    results = runner.run_single_test(
        TestType.EXPLORATION_BIAS,
        test_values=[1.0, 1.4, 2.0],
        games_per_test=6  # Very small for demo
    )

    print("\n✅ Demo test completed!")
    return results

def run_research_replication():
    """Run the full research methodology replication"""
    runner = TournamentRunner()
    return runner.run_all_tests(quick_mode=False)

def run_quick_analysis():
    """Run a quick version for testing purposes"""
    runner = TournamentRunner()
    return runner.run_all_tests(quick_mode=True)

if __name__ == "__main__":
    import sys

    print("🏆 CHESS ENGINE TESTING FRAMEWORK")
    print("=" * 50)
    print("Available commands:")
    print("  python testing_framework.py demo     - Quick demo")
    print("  python testing_framework.py quick    - Quick analysis")
    print("  python testing_framework.py full     - Full research replication")
    print("  python testing_framework.py custom   - Custom matchup")
    print("=" * 50)

    if len(sys.argv) > 1:
        command = sys.argv[1].lower()

        if command == "demo":
            demo_quick_test()
        elif command == "quick":
            run_quick_analysis()
        elif command == "full":
            run_research_replication()
        elif command == "custom":
            # Example custom matchup
            runner = TournamentRunner()
            config1 = {
                'engine_type': 'MCTS',
                'evaluation_level': EvaluationLevel.ADVANCED,
                'time_limit': 2.0,
                'exploration_constant': 1.4
            }
            config2 = {
                'engine_type': 'ABP',
                'evaluation_level': EvaluationLevel.ADVANCED,
                'max_depth': 5
            }
            runner.run_custom_matchup(config1, config2, games=20)
        else:
            print(f"Unknown command: {command}")
    else:
        print("\nNo command specified. Use 'demo' for a quick test.")
        print("Example: python testing_framework.py demo")

🏆 CHESS ENGINE TESTING FRAMEWORK
Available commands:
  python testing_framework.py demo     - Quick demo
  python testing_framework.py quick    - Quick analysis
  python testing_framework.py full     - Full research replication
  python testing_framework.py custom   - Custom matchup
Unknown command: -f


In [ ]:
run_research_replication()

Streaming output truncated to the last 5000 lines.
   Node h3h1: visits=1, wins=110.00, win_rate=110.000
   Node ROOT: visits=6, wins=-350.00, win_rate=-58.333
   ✅ Backpropagated through 2 nodes

🔄 ITERATION 7
----------------------------------------
🌱 PHASE 2 - EXPANSION:
   ✅ Expanded with move: d2e3
   New child visits: 0, terminal: False
🎲 PHASE 3 - SIMULATION:
   Starting rollout from position
   🎯 Rollout complete: 100 moves, result=450.000
⬆️  PHASE 4 - BACK-PROPAGATION:
   Propagating result: 450.000
   Node d2e3: visits=1, wins=450.00, win_rate=450.000
   Node ROOT: visits=7, wins=-800.00, win_rate=-114.286
   ✅ Backpropagated through 2 nodes

🔄 ITERATION 8
----------------------------------------
🌱 PHASE 2 - EXPANSION:
   ✅ Expanded with move: d2d1
   New child visits: 0, terminal: False
🎲 PHASE 3 - SIMULATION:
   Starting rollout from position
   🎯 Rollout complete: 39 moves, result=1.000
⬆️  PHASE 4 - BACK-PROPAGATION:
   Propagating result: 1.000
   Node d2d1: visits=1, w